In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.
print(os.listdir("../input/twitter-senti"))
print(os.listdir("../input/glove-twitter"))

['twitter-senti', 'glove-twitter']
['train.tsv', 'test.tsv']
['glove.twitter.27B.25d.txt']


In [2]:

train = pd.read_csv('../input/twitter-senti/train.tsv',sep='\t')
test = pd.read_csv('../input/twitter-senti/test.tsv',sep='\t')
train.head()

,id,tweet,label
0,264183816548130816,Gas by my house hit $3.39!!!! I'm going to Cha...,1
1,264249301910310912,Iranian general says Israel's Iron Dome can't ...,-1
2,264105751826538497,with J Davlar 11th. Main rivals are team Polan...,1
3,264094586689953794,"Talking about ACT's && SAT's, deciding where I...",-1
4,254941790757601280,"They may have a SuperBowl in Dallas, but Dalla...",-1


In [3]:
test.head()

,id,tweet,label
0,260097528899452929,"Won the match #getin . Plus, tomorrow is a ver...",0
1,263791921753882624,Some areas of New England could see the first ...,0
2,260486470828171265,Tina Fey & Amy Poehler are hosting the Golden ...,0
3,262968617233162240,Lunch from my new Lil spot ...THE COTTON BOWL ...,1
4,263790847424880641,SNC Halloween Pr. Pumped. Let's work it for Su...,1


In [4]:
embed_matrix = {}
file= open('../input/glove-twitter/glove.twitter.27B.25d.txt')
for line in file:
    values = line.split()
    word = values[0]
    coefs = np.array(values[1:],dtype='float32')
    embed_matrix[word]=coefs
file.close()

In [5]:
len(embed_matrix)

1193515

In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7589 entries, 0 to 7588
Data columns (total 3 columns):
id       7589 non-null int64
tweet    7589 non-null object
label    7589 non-null int64
dtypes: int64(2), object(1)
memory usage: 177.9+ KB


In [7]:
train.label.unique()

array([ 1, -1,  0])

In [8]:
from  keras.preprocessing.text import Tokenizer
token = Tokenizer(num_words=2000, split=' ')
token.fit_on_texts(list(train['tweet']))
X = token.texts_to_sequences(list(train['tweet']))

Using TensorFlow backend.


In [9]:
length =[len(x) for x in X]
print (max(length))

175


In [10]:
from keras.preprocessing.sequence import pad_sequences
X_pad = pad_sequences(X)   # Pad sequences with maximum length

In [11]:
X_pad.shape[1]

175

In [12]:
Y = pd.get_dummies(train['label'])
Y.shape

(7589, 3)

In [13]:
from keras.models import Sequential
from keras.layers import Dense,Activation,Flatten,Dropout,BatchNormalization

In [14]:
model = Sequential()
model.add(Dense(128,input_dim=X_pad.shape[1],activation='relu'))
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(Dense(256,activation='relu'))
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(Dense(3,activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 128)               22528     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 128)               512       
_________________________________________________________________
dense_2 (Dense)              (None, 256)               33024     
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 256)               1024      
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 771       
Total para

In [15]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(X_pad,Y, test_size=0.2,random_state = 100)

In [16]:
x_train.shape

(6071, 175)

In [17]:
x_test.shape

(1518, 175)

In [18]:
model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])
model.fit(x_train,y_train,epochs= 50,batch_size=128)

Epoch 1/50
6071/6071 [==============================] - 2s 287us/step - loss: 1.4909 - acc: 0.3515
Epoch 2/50
6071/6071 [==============================] - 0s 58us/step - loss: 1.3539 - acc: 0.3820
Epoch 3/50
6071/6071 [==============================] - 0s 55us/step - loss: 1.2695 - acc: 0.3914
Epoch 4/50
6071/6071 [==============================] - 0s 58us/step - loss: 1.2094 - acc: 0.4113
Epoch 5/50
6071/6071 [==============================] - 0s 59us/step - loss: 1.1676 - acc: 0.4185
Epoch 6/50
6071/6071 [==============================] - 0s 55us/step - loss: 1.1449 - acc: 0.4261
Epoch 7/50
6071/6071 [==============================] - 0s 56us/step - loss: 1.1148 - acc: 0.4274
Epoch 8/50
6071/6071 [==============================] - 0s 57us/step - loss: 1.0902 - acc: 0.4398
Epoch 9/50
6071/6071 [==============================] - 0s 57us/step - loss: 1.0688 - acc: 0.4355
Epoch 10/50
6071/6071 [==============================] - 0s 56us/step - loss: 1.0725 - acc: 0.4383
Epoch 11/50
6071/6

In [19]:
model.evaluate(x_test,y_test)

1518/1518 [==============================] - 0s 120us/step


[0.996126765633132, 0.4822134386566474]

In [20]:
diction = token.word_index
len(diction)

21048

In [21]:
EMBEDDING_DIM = 25
embedding_matrix = np.zeros((len(token.word_index) + 1, EMBEDDING_DIM))
for word, i in token.word_index.items():
    embedding_vector = embed_matrix.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [22]:
embedding_matrix.shape

(21049, 25)

In [23]:
VOC_SIZE = len(token.word_index)+1
EMB_DIM = 25
INPUT_DIM = X_pad.shape[1]
from keras.layers import Embedding, SimpleRNN,SpatialDropout1D,LSTM,GRU
from keras import regularizers
from keras.optimizers import *
rnn_model = Sequential()
rnn_model.add(Embedding(VOC_SIZE, EMB_DIM, input_length=INPUT_DIM,weights=[embedding_matrix],trainable=False))
rnn_model.add(SpatialDropout1D(0.4))
rnn_model.add(SimpleRNN(64,activation='relu',dropout=0.5, recurrent_dropout=0.5))
rnn_model.add(Dropout(0.5))
rnn_model.add(Dense(3,activation='softmax',kernel_regularizer=regularizers.l2(0.01)))
rnn_model.compile(loss='categorical_crossentropy', optimizer=Adam(0.01),metrics=['accuracy'])
rnn_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 175, 25)           526225    
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 175, 25)           0         
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 64)                5760      
_________________________________________________________________
dropout_3 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 3)                 195       
Total params: 532,180
Trainable params: 5,955
Non-trainable params: 526,225
_________________________________________________________________


In [24]:
rnn_model.fit(x_train,y_train,epochs=5,batch_size=128)

Epoch 1/5
6071/6071 [==============================] - 7s 1ms/step - loss: 1.4389 - acc: 0.4474
Epoch 2/5
6071/6071 [==============================] - 7s 1ms/step - loss: 1.0185 - acc: 0.4902
Epoch 3/5
6071/6071 [==============================] - 7s 1ms/step - loss: 1.0103 - acc: 0.4802
Epoch 4/5
6071/6071 [==============================] - 7s 1ms/step - loss: 0.9996 - acc: 0.4879
Epoch 5/5
6071/6071 [==============================] - 7s 1ms/step - loss: 0.9980 - acc: 0.4848


In [25]:
rnn_model.evaluate(x_test,y_test)

1518/1518 [==============================] - 4s 2ms/step


[0.9880187464325796, 0.48287220049909607]

In [26]:
lstm_model = Sequential()
lstm_model.add(Embedding(VOC_SIZE, EMB_DIM, input_length=INPUT_DIM,weights=[embedding_matrix],trainable=False))
lstm_model.add(LSTM(64,activation='relu'))
lstm_model.add(Dense(3,activation='softmax'))
lstm_model.compile(loss='categorical_crossentropy', optimizer=Adam(0.001),metrics=['accuracy'])
lstm_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 175, 25)           526225    
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                23040     
_________________________________________________________________
dense_5 (Dense)              (None, 3)                 195       
Total params: 549,460
Trainable params: 23,235
Non-trainable params: 526,225
_________________________________________________________________


In [ ]:
lstm_model.fit(x_train,y_train,epochs=5,batch_size=128)

In [ ]:
lstm_model.evaluate(x_test,y_test)

In [ ]:
gru_model = Sequential()
gru_model.add(Embedding(VOC_SIZE, EMB_DIM, input_length=INPUT_DIM,weights=[embedding_matrix],trainable=False))
gru_model.add(GRU(64,activation='relu'))
gru_model.add(Dense(3,activation='softmax'))
gru_model.compile(loss='categorical_crossentropy', optimizer=Adam(0.001),metrics=['accuracy'])
gru_model.summary()

In [ ]:
gru_model.fit(x_train,y_train,epochs=10,batch_size=128)

In [ ]:
gru_model.evaluate(x_test,y_test)